# Optuna Training

Optuna is a powerful tool for performing hyperparameter optimization. It is fast and surprisingly simple to use with options for much more advanced customization. This tutorial will focus on the basic syntax for using Optuna.

In [1]:
import optuna

import numpy as np

from sklearn import datasets
from sklearn import svm
from sklearn import linear_model
from sklearn import model_selection

All of the code we would normally use for model training goes in a function called `objective` that takes an Optuna-specific argument called `trial`. The `objective` function must have the following within it:  

1. Definition of `X` and `y`
2. Selection of model hyperparameters using a `suggest_int`, `suggest_float`, `suggest_categorical`, or some other `suggest_` function.
3. Definition of the model using the selected hyperparameters
4. Generation of a model score  

See the following code for examples of the syntax Optuna uses.

In [6]:
def objective(trial):
    """This is the set of instructions for Optuna to follow. 
    Everything that you would normally do to train an algorithm goes in this function."""
    # Define the dataset, X and y
    diabetes = datasets.load_diabetes()
    X, y = diabetes.data, diabetes.target
    # Pick a model type
    model_name = trial.suggest_categorical("model_type", ["SVM", "Linear"])
    
    # Get the hyperparameters based on the model type
    
    # SVM
    if model_name == "SVM":
        # Get the hyperparameters from a given range
        kern = trial.suggest_categorical("SVM_kernel", ["linear", "poly", "rbf", "sigmoid"])
        C = trial.suggest_float("SVM_C", 0.01, 100, log=True)
        if kern == "poly":
            deg = trial.suggest_int("SVM_degree", 2, 10, log=False)
        else:
            deg = 3
        # Define the model with the chosen parameters
        model = svm.SVR(kernel=kern, C=C, degree=deg)
    
    # Linear
    elif model_name == "Linear":
        # Get the hyperparameters from a given range
        a = trial.suggest_float("Linear_alpha", 0.001, 100, log=True)
        ratio = trial.suggest_float("Linear_l1_ratio", 0, 1, log=False)
        # Define the model with the chosen parameters
        model = linear_model.ElasticNet(alpha=a, l1_ratio=ratio)
    
    # Get the negative mean squared error score with cross validation
    scores = model_selection.cross_val_score(model, X, y, cv=5, scoring="neg_mean_squared_error")
    score = np.mean(scores)
    # Return the score
    return score

After defining the function called `objective`, only two lines are required to use Optuna for hyperparameter optimization. We need to _create a study_ by calling `study = optuna.create_study()`, then we need to optimize the study to get the best hyperparameters by calling `study.optimize(objective)`. We can use or print the best hyperparameters after this.

In [7]:
# Create the study to maximize the negative mean squared error
study = optuna.create_study(direction="maximize")
# Optimize the study to get the best hyperparameters
study.optimize(objective, n_trials=100)
# Show the study results
print("MSE: "+str(-study.best_trial.values[0]))
print("Best parameters: "+str(study.best_trial.params))

[I 2022-10-17 16:45:20,356] A new study created in memory with name: no-name-e97514c8-121a-4e31-8b0c-04ffedece628
[I 2022-10-17 16:45:20,385] Trial 0 finished with value: -5894.214139555121 and parameters: {'model_type': 'Linear', 'Linear_alpha': 0.5425638953395514, 'Linear_l1_ratio': 0.28241794327152825}. Best is trial 0 with value: -5894.214139555121.
[I 2022-10-17 16:45:20,409] Trial 1 finished with value: -5882.565756963317 and parameters: {'model_type': 'Linear', 'Linear_alpha': 0.39271213498494795, 'Linear_l1_ratio': 0.04035017235170135}. Best is trial 1 with value: -5882.565756963317.
[I 2022-10-17 16:45:20,433] Trial 2 finished with value: -5982.413413836098 and parameters: {'model_type': 'Linear', 'Linear_alpha': 50.49564710722571, 'Linear_l1_ratio': 0.6148953089674932}. Best is trial 1 with value: -5882.565756963317.
[I 2022-10-17 16:45:20,455] Trial 3 finished with value: -5975.733372439451 and parameters: {'model_type': 'Linear', 'Linear_alpha': 3.3806792968884434, 'Linear_

MSE: 2992.258831219395
Best parameters: {'model_type': 'Linear', 'Linear_alpha': 0.0021086436483865557, 'Linear_l1_ratio': 0.99981440637672}


In [ ]:
def objective_demonstration(trial):
    """This is the set of instructions for Optuna to follow. 
    Everything that you would normally do to train an algorithm goes in this function."""
    # Define the dataset, X and y
    # Pick a model type
    # Get the hyperparameters based on the model type
        # Get the hyperparameters from a given range
    # Get the score with cross validation
    # Return the score

In [ ]:
# Create the study
# Optimize the study
# Show the study results